In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [2]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = '!#$%&\'*+:;<=>?[\\]^`{|}~'
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [3]:
### load NER model best
model_ner = spacy.load('./output/model-best/')

In [4]:
# image = cv2.imread('./allinvoices_img/003.jpg')

In [5]:
# load Image
image = cv2.imread('./allinvoices_img_2/077.jpg')

# cv2.imshow('businesscard', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# extract data using pytesseract

tessData = pytesseract.image_to_data(image, lang='deu')
# convert into Dataframe
tessList = list(map(lambda x: x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:], columns=tessList[0])
df.dropna(inplace=True)
df['conf'] =df['conf'].astype(float).astype(int)
df['text'] =df['text'].apply(cleanText)
#print(df)

# convert data into content
df_clean = df.query("text != '' ")
content = " ".join([w for w in df_clean['text']])
print(content)

# get prediction from NER model
doc = model_ner(content)


kaspersky NSAwar RECHNUNG Paris, 2021-10-16 Nexway SAS CS 70002 30 020 Nimes Cedex 1, FRANCE Fax 33 4 66 38 03 28 Rechnungsadresse RECHNUNGSNUMMER LEON BABUCKE FR-21-817012571 RECHNUNGSDATUM Eichbuschallee 1 2021-10-16 72138 Kirchentellinsfurt, Deutschland Bestell-Nummer email Leon.Babucke@gmx.de 6UR9I9WFYFLI Zahlungsweise VISA die Produkte von Kaspersky DACH Nexway werden von Nexway vertrieben. Bei einer Zahlung mit Kreditkarte wird auf Ihrem Kontoauszug Kaspersky DACH Nexway ausgewiesen. Ihre erworbenen Produkte Referenz OW3638VCHLZF Produkt League of Legends Account EUW 40-50k BE Lol Smurf Lvl 30 10,92 € 19 10,92 € UNVERIFIED Unranked 10,92 € MwSt. (19) 2,07 € Hauptsitz Nexway SAS, 1 avenue du General de Gaulle Tour PB5 92074 Paris La Defense Cedex France TVA FR33440953859 - CH CHE-190.925.697 - AU 300 015 521 065 SIRET 44095385900053 APE 6209Z SIREN 440953859 R.C.S NANTERRE SAS au


In [6]:
from spacy import displacy

In [7]:
displacy.render(doc,style='ent', jupyter=True)

# Tagging

In [8]:
docjson = doc.to_json()
docjson.keys()
# docjson

dict_keys(['text', 'ents', 'tokens'])

In [9]:
doc_text = docjson['text']
doc_text

'amazon.de Rechnung Zahlungsreferenznummer 3NiPoAZm4SUm9qBoGKDi Verkauft von Amazon EU S.är.l., Niederlassung Deutschland USt-IDNr. DE8314584193 Rechnungsdatum /Lieferdatum 05 May 2019 AAMU ABEN Rechnungsnummer DE3MQ236AEUI Radickestraße 25 Zahlbetrag 359€ Jettingen, 71131 Germany Um unseren Kundenservice zu kontaktieren, besuchen Sie www.amazon.de/contact-us Rechnungsadresse Lieferadresse Verkauft von AAMU ABEN AAMU ABEN Amazon EU 5S.ä r.l., Niederlassung Deutschland Radickestraße 25 Radickestraße 25 Marcel-Breuer-Str. 12 Jettingen, 71131 Jettingen, 71131 80807 München Germany Germany Deutschland USt-IDNr. DE8145384193 Bestellinformationen Bestelldatum 02 May 2019 Bestellnummer 123-1233703-1233703 Rechnungsdetails Beschreibung Menge Stückpreis USt. Stückpreis Zwischensumme (ohne USt.) (inkl. USt.) (inkl. USt.) Rost Kerze Edelrost 1 3,02 € 19 3,59€ 3,59€ ASIN if492jf236 Versandkosten 0,00 € 0,00 € 0,00 € Gesamtpreis 3,59€ USt. Zwischensumme USt. (ohne USt.) 19 3,02 € 0,57€ USt. Gesamt 

In [10]:
dataFrame_tokens = pd.DataFrame(docjson['tokens'])
dataFrame_tokens['tokens'] = dataFrame_tokens[['start', 'end']].apply(lambda x: doc_text[x[0]:x[1]], axis= 1)

In [11]:
# dataFrame_tokens[:50]

In [12]:
doc_text[0:9]

'amazon.de'

In [13]:
right_table = pd.DataFrame(docjson['ents'])[['start', 'label']]
dataFrame_tokens = pd.merge(dataFrame_tokens, right_table, how='left', on='start')

In [14]:
# dataFrame_tokens.head(10)

In [15]:
dataFrame_tokens.fillna('O', inplace=True)
dataFrame_tokens.head(10)

,id,start,end,tokens,label
0,0,0,9,amazon.de,B-ORG
1,1,10,18,Rechnung,O
2,2,19,41,Zahlungsreferenznummer,O
3,3,42,62,3NiPoAZm4SUm9qBoGKDi,O
4,4,63,71,Verkauft,O
5,5,72,75,von,O
6,6,76,82,Amazon,O
7,7,83,85,EU,O
8,8,86,92,S.är.l,O
9,9,92,93,.,O


In [16]:
# join label to df_clean dataFrame
df_clean['end'] = df_clean['text'].apply(lambda x: len(x) + 1).cumsum() - 1
df_clean['start'] = df_clean[['text', 'end']].apply(lambda x: x[1] - len(x[0]), axis=1)

C:\Users\Admin\AppData\Local\Temp/ipykernel_16420/2344595886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['end'] = df_clean['text'].apply(lambda x: len(x) + 1).cumsum() - 1
C:\Users\Admin\AppData\Local\Temp/ipykernel_16420/2344595886.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['start'] = df_clean[['text', 'end']].apply(lambda x: x[1] - len(x[0]), axis=1)


In [17]:
# df_clean[:50]

In [18]:
# inner join with start
dataframe_info = pd.merge(df_clean, dataFrame_tokens[['start', 'tokens', 'label']], how='inner', on='start')

In [19]:
# dataframe_info

# Bounding Box

In [20]:
bb_df = dataframe_info.query("label != 'O' ")
# bb_df

In [23]:
img = image.copy()
for x,y,w,h,label in bb_df[['left', 'top', 'width', 'height', 'label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img, (x,y), (x+w, y+h), (64, 255, 0), 2)
#     cv2.putText(img, str(label), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 0, 0), 6,cv2.LINE_AA, False)
#     cv2.putText(img, str(label), (x+42, y+42), cv2.FONT_HERSHEY_PLAIN, 5, (255, 0, 0), 3)
    cv2.putText(img, str(label), (x, y), cv2.FONT_HERSHEY_PLAIN, 5, (255, 0, 0), 3)

cv2.namedWindow("Predictions", cv2.WINDOW_NORMAL)
imS = cv2.resize(img, (4136, 5848))    
cv2.imshow('Predictions', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# bb_df['label'][1][2:]
# bb_df.head(10)

In [25]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
# bb_df.head(10)

C:\Users\Admin\AppData\Local\Temp/ipykernel_16420/2923300729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])


In [26]:
# group the labels
class groupgen():
    def __init__(self):
        self.id = 0
        self. text = ''
        
    def get_group(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen() 

In [27]:
bb_df['group'] = bb_df['label'].apply(grp_gen.get_group)
# bb_df.head()

C:\Users\Admin\AppData\Local\Temp/ipykernel_16420/2283708908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['group'] = bb_df['label'].apply(grp_gen.get_group)


In [28]:
# right and bottom of bounding box
bb_df[['left', 'top', 'width', 'height']] = bb_df[['left', 'top', 'width', 'height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

C:\Users\Admin\Desktop\InvoiceReader_Project\InvoiceReader\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Admin\AppData\Local\Temp/ipykernel_16420/3100329007.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['right'] = bb_df['left'] + bb_df['width']
C:\Users\Admin\AppData\Local\Temp/ipykernel_16420/3100329007.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [29]:
# Tagging: groupby group
col_group = ['left', 'top', 'right', 'bottom', 'label', 'tokens', 'group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [30]:
print(group_tag_img)

In [31]:
img_tagging = group_tag_img.agg({
   
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'tokens':lambda x: " ".join(x)
})

In [32]:
img_tagging

,left,right,top,bottom,label,tokens
group,,,,,,
1,219,898,18,197,ORG,amazon.de
2,3116,3171,977,1016,DATUM,05
3,436,611,1045,1091,ADRESSE,AAMU
4,633,2841,1045,1124,NAME,ABEN Rechnungsnummer
5,3118,3503,1068,1124,RNUM,DE3MQ236AEUI
6,3116,3256,1180,1217,BETRAG,359
7,439,887,1231,1383,ADRESSE,71131 Germany
8,3676,3880,3659,3726,BETRAG,"3,59"


In [33]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb, (l,t), (r,b), (0,255,0), 2)
    
    cv2.putText(img_bb, label, (l-350,t+42), cv2.FONT_HERSHEY_PLAIN, 6, (255, 0, 0), 3)

cv2.namedWindow("Bounding Box Invoice", cv2.WINDOW_NORMAL)
imS = cv2.resize(img_bb, (8136, 10848))    
cv2.imshow('Bounding Box Invoice', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
def parser(text,label):
    if label == 'BETRAG':
#         text = text.lower()
#         text = re.sub(r'\D', '', text)
        text = text
    
    elif label in ('RNUM'):
        text = text
        
    elif label == 'DATUM':
#         text = text.lower()
#         allow_special_char = '-./'
#         text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char), '', text)
        text = text
            
    elif label == 'ADRESSE':
#         text = text.lower()
        allow_special_char = '-.()ßÜüÖöÄä'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char), '', text)
    
    elif label in ('NAME'):
#         text = re.sub(r'[^A-Za-z ]', '', text)
#         text = text.title()
        text = text
    
    elif label == 'ORG':
#         text = text.lower()
        text = re.sub(r'[^A-Za-z0-9. ]', '', text)
#         text = text.title()
    
    return text

In [35]:
# test the parser
# parser('omid12)&#$@gmail.com', 'EMAIL')

In [36]:
info_array = dataframe_info[['tokens', 'label']].values
entities = dict(NAME=[], ORG=[], DATUM=[], ADRESSE=[], BETRAG=[], RNUM=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]
    
    # step -1 parse the token
    text = parser(token, label_tag)
    
    if bio_tag in ('B', 'I'):
        
        if previous != label_tag:
            entities[label_tag].append(text)
        
        else:
            if bio_tag == "B":
                entities[label_tag].append(text)
                
            else:
                if label_tag in ("NAME", 'ORG', 'DATUM', 'ADRESSE', 'BETRAG', 'RNUM'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
    
    
    previous = label_tag
    

In [37]:
print(entities)

{'NAME': ['ABEN Rechnungsnummer'], 'ORG': ['amazon.de'], 'DATUM': ['05'], 'ADRESSE': ['AAMU', '71131 Germany'], 'BETRAG': ['359', '3,59'], 'RNUM': ['DE3MQ236AEUI']}
